In [ ]:
!nvidia-smi

Sun Nov  7 15:46:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.9 MB/s eta 0:00:26tcmalloc: large alloc 1147494400 bytes == 0x5595f791e000 @  0x7f9375a99615 0x5595bed794cc 0x5595bee5947a 0x5595bed7c2ed 0x5595bee6de1d 0x5595bedefe99 0x5595bedea9ee 0x5595bed7dbda 0x5595bedefd00 0x5595bedea9ee 0x5595bed7dbda 0x5595bedec737 0x5595bee6ec66 0x5595bedebdaf 0x5595bee6ec66 0x5595bedebdaf 0x5595bee6ec66 0x5595bedebdaf 0x5595bed7e039 0x5595bedc1409 0x5595bed7cc52 0x5595bedefc25 0x5595bedea9ee 0x5595bed7dbda 0x5595bedec737 0x5595bedea9ee 0x5595bed7dbda 0x5595bedeb915 0x5595bed7dafa 0x5595bedebc0d 0x5595bedea9ee
     |████████████████████████████████| 881.9 MB 19 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.10.0 which is incompatible.
torchtext 0.10.0 re

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 598041763 bytes (571 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip
--
Path = /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip
Type = zip
Physical Size = 598041763

  0%      1% 13 - dataset_pat_bandPass2/191_2b1_Pr_mc_LittC2SE_bandPass2.wav                                                                      1% 20 - dataset_pat_bandPass2/146_2b4_Ar_mc_AKGC417L_bandPass2.wav

In [ ]:
train_pathology = pd.read_csv('/content/drive/MyDrive/Tesi/train_binary_pathology.csv')
test_pathology = pd.read_csv('/content/drive/MyDrive/Tesi/test_binary_pathology.csv')

In [ ]:
train_pathology

,filename,label
0,/content/dataset_pat_bandPass2/205_1b3_Ar_mc_A...,0
1,/content/dataset_pat_bandPass2/151_3p2_Al_mc_A...,0
2,/content/dataset_pat_bandPass2/226_1b1_Ll_sc_M...,0
3,/content/dataset_pat_bandPass2/130_3p2_Pl_mc_A...,0
4,/content/dataset_pat_bandPass2/135_2b2_Al_mc_L...,0
...,...,...
731,/content/dataset_pat_bandPass2/186_3b3_Tc_mc_A...,0
732,/content/dataset_pat_bandPass2/112_1p1_Pl_sc_L...,0
733,/content/dataset_pat_bandPass2/138_1p3_Pl_mc_A...,0
734,/content/dataset_pat_bandPass2/197_1b1_Al_sc_M...,0


In [ ]:
train_multiclass = train_pathology.copy()

In [ ]:
test_multiclass = test_pathology.copy()

In [ ]:
train_multiclass.groupby('label').count()

,filename
label,
0,711
1,25


In [ ]:
test_multiclass.groupby('label').count()

,filename
label,
0,174
1,10


In [ ]:
!pip -q install torchaudio
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 31.6 MB/s 
     |████████████████████████████████| 596 kB 35.0 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 3.3 MB 34.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torchaudio
duration = 20
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result
        
        return result

In [ ]:
train_ds = AudiosDataset(train_multiclass['filename'], train_multiclass['label'])
test_ds = AudiosDataset(test_multiclass['filename'], test_multiclass['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 2
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pip install fairseq@git+https://github.com//pytorch/fairseq.git@f2146bdc7abf293186de9449bfa2272775e39e1d#egg=fairseq
!pip install sox

  Cloning https://github.com//pytorch/fairseq.git (to revision f2146bdc7abf293186de9449bfa2272775e39e1d) to /tmp/pip-install-yxlbh2xl/fairseq_64ed8ae4632b489d943e932464510085
  Running command git clone -q https://github.com//pytorch/fairseq.git /tmp/pip-install-yxlbh2xl/fairseq_64ed8ae4632b489d943e932464510085
  Running command git rev-parse -q --verify 'sha^f2146bdc7abf293186de9449bfa2272775e39e1d'
  Running command git fetch -q https://github.com//pytorch/fairseq.git f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git checkout -q f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 6.6 M

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModel
DEVICE = 'cuda'
m = AutoModel.from_pretrained("ntu-spml/distilhubert")

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for aalbert classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.out_proj = nn.Linear(512, 2)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = features[0]
        res = torch.mean(features, dim=1)
        logit = self.classifier(res)
        return logit

In [ ]:
model = Classifier(m)

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime


def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+2}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/DistilHubert/final_80-20_NO-BALANCED_bandPass2_dropout=0.5_PATHOLOGY_healthy-unhealthy_distilhubert_lr={lr}_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

In [ ]:
freq=16
duration=20
train_ds = AudiosDataset(train_multiclass['filename'], train_multiclass['label'])
test_ds = AudiosDataset(test_multiclass['filename'], test_multiclass['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 2
num_workers = 2
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

------------
Modello finale

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("ntu-spml/distilhubert")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 HUBERT FINAL

* Epoch 1/100


Loss: 0.07 #nan->0: 100%|██████████| 368/368 [04:13<00:00,  1.45it/s]


loss: 0.208059
acc: 0.955163
start validation


Loss: 1.61 #nan->0: 100%|██████████| 92/92 [00:22<00:00,  4.10it/s]


validation loss: 0.19706332683563232
validation accuracy: 0.9456521739130435
Save new model!- 7/11 h17:58
--------------------
* Epoch 2/100


...

validation loss: 0.10150355845689774
validation accuracy: 0.9510869565217391
--------------------
* Epoch 12/100


Loss: 0.00 #nan->0: 100%|██████████| 368/368 [04:13<00:00,  1.45it/s]


loss: 0.032346
acc: 0.989130
start validation


Loss: 0.12 #nan->0: 100%|██████████| 92/92 [00:22<00:00,  4.09it/s]

validation loss: 0.13225384056568146
validation accuracy: 0.9565217391304348
--------------------
